# RL solutions for combinatorial games

## The Game of Nim(Mathematical jargon):
- The Game of Nim is a simple 2 player impartial perfect information game
- Any Combinatorial Game is equivalent to a one heap game of nim under normal play $\rightarrow$ [Sprague Grundy Theorem](https://en.wikipedia.org/wiki/Sprague%E2%80%93Grundy_theorem)
- refer [MIT lecture notes](https://web.mit.edu/sp.268/www/nim.pdf) or [wikipedia](https://en.wikipedia.org/wiki/Nim) for more

## The Game of Nim(In simpler terms):
- The game of Nim is given by the following setting:
- There is a set of **heaps** arranged in front of you each containing some number of **pebbles**
- On your turn you must pick a non-empty heap and take out any non-zero number of pebbles from it and obviously you cannot take out more pebbles than there are in the heap
- There are 2 ways to play Nim which are "equivalent"(quotes here because the notation of equivalence here is clearly defined but omitted here)
    1. Normal Play: The player who empties the last heap wins
    2. Misère Play: The player who is forced to take the last pebble wins

### The winning strategy:
- It can be proved that this game has a winning strategy which is to always end your turn on a **Nim Sum** of zero
- For the case of our simple nim game, the nim sum is simply the XOR of the number of pebbles in each heap

Note: Under certain conditions, **Go** endgames can be modeled as a combinatorial game

In [49]:
import numpy as np
from tqdm import tqdm

In [50]:
np.random.seed(42)

In [51]:
# A Simple Nim Game
class SimpleNim:
    def __init__(self, state):
        self.heaps = state
        self.turn = 0
        self.num_heaps = len(self.heaps)

    def get_state(self):
        if isinstance(self.heaps, list):
            return tuple(self.heaps)
        else:
            return tuple(self.heaps.tolist())

    def get_num_heaps(self):
        return self.num_heaps

    def get_heap_size(self, heap_index):
        if 0 <= heap_index < self.num_heaps:
            return self.heaps[heap_index]
        return None

    def is_game_over(self):
        return all(pebbles == 0 for pebbles in self.heaps)

    def make_move(self, heap_index, num_pebbles):
        if self.is_game_over():
            return False

        # Validate the move
        if 0 <= heap_index < self.num_heaps and 0 < num_pebbles <= self.heaps[heap_index]:
            self.heaps[heap_index] -= num_pebbles
            self.turn = 1 - self.turn  # Switch turns
            return True
        return False

    def winner(self):
        if self.is_game_over():
            return self.turn
        return None

In [52]:
def play_optimal(game):
    if game.is_game_over():
        return
    heaps = game.get_state()

    nim_sum = 0
    largest = -1
    largest_index = -1
    for i, heap in enumerate(heaps):
        nim_sum ^= heap
        if heap > largest:
            largest = heap
            largest_index = i
    # print(nim_sum)
    if nim_sum != 0:
        game.make_move(largest_index, largest - (largest^nim_sum)) # optimal move
    else:
        game.make_move(largest_index, 1) # take the move that progresses the game to the least possible extent and hope for a blunder

In [53]:
# simple simulation of a nim game with both players using the optimal strategy
heaps = [10] * 3
game = SimpleNim(heaps)
while not game.is_game_over():
    play_optimal(game)
    print(game.get_state())
print("The winner is Player ", game.winner())

(0, 10, 10)
(0, 9, 10)
(0, 9, 9)
(0, 8, 9)
(0, 8, 8)
(0, 7, 8)
(0, 7, 7)
(0, 6, 7)
(0, 6, 6)
(0, 5, 6)
(0, 5, 5)
(0, 4, 5)
(0, 4, 4)
(0, 3, 4)
(0, 3, 3)
(0, 2, 3)
(0, 2, 2)
(0, 1, 2)
(0, 1, 1)
(0, 0, 1)
(0, 0, 0)
The winner is Player  1


## Defining State and Action Spaces:
**The State** is given by a configuration of heaps($\N_0$ means the set of naturals that includes zero):
$$ \mathcal S = \N_0^h $$
where h is the number of heaps
**Actions** are given by two tuples of heap number and number of pebbles to be taken out:
$$ \mathcal A(s) = H(s) \times \N $$ 
where $\N$ does not include zero and $H(s)$ is the set of heaps having non zero number of pebbles in the state s

In [54]:
# for simplicity consider the version of the problem that starts off with same number of pebbles in each heap and 3 heaps
num_heaps = 3
initial_pebbles = 5
max_episodes = 100000

In [55]:
class EpsilonGreedyPolicy:
    def __init__(self):
        self.epsilon = 1

    def update(self, epsilon):
        self.epsilon = epsilon

    def policy(self, s, Q):
        if s not in Q or np.random.random() < self.epsilon:
            index = np.random.randint(0, len(s))
            while s[index] == 0:
                index = np.random.randint(0, len(s))
            return (index, 1)
        else:
            return max(Q[s], key=Q[s].get)

    def final_policy(self, s, Q):
        if s not in Q:
            return (0,0)
        return max(Q[s], key=Q[s].get)

In [56]:
class SimpleGreedyPolicy:
    def update(self, epsilon): pass

    def policy(self, s, Q):
        if s not in Q:
            index = np.random.randint(0, len(s))
            while s[index] == 0:
                index = np.random.randint(0, len(s))
            return (index, 1)
        return max(Q[s], key=Q[s].get)

    def final_policy(self, s, Q):
        if s not in Q:
            return (0,0)
        return max(Q[s], key=Q[s].get)

In [57]:
def generate_episode(policy, s0, a0, Q):

    # init
    game = SimpleNim(s0)
    states = []
    actions = []
    rewards = []

    first_state = tuple(s0.tolist())
    states.append(first_state)
    actions.append(a0)
    game.make_move(a0[0], a0[1])

    first_visit = {(first_state,a0): 0}
    index = 1

    # loop until end of episode
    while not game.is_game_over():
        # opponent move
        play_optimal(game)

        # agent move
        if game.is_game_over():
            break
        rewards.append(0)
        curr_state = game.get_state()
        states.append(curr_state)
        curr_action = policy.policy(curr_state, Q)
        actions.append(curr_action)
        game.make_move(curr_action[0], curr_action[1])

        if (tuple(curr_state), curr_action) not in first_visit:
            first_visit[(tuple(curr_state), curr_action)] = index
        index+=1

    if game.winner() == 1:
        rewards.append(1)
    else:
        rewards.append(0)
    return states, actions, rewards, first_visit

In [58]:
# Monte Carlo Control with exploring starts
def monte_carlo_es(gamma=1, policy=EpsilonGreedyPolicy()):
    Q = {} # take default to be zero
    N = {}

    for i in tqdm(range(max_episodes)):
        s0 = np.random.randint(0, initial_pebbles+1, num_heaps)
        while all(s0 == 0):
            s0 = np.random.randint(0, initial_pebbles+1, num_heaps)
        heap_num = np.random.randint(0,num_heaps)
        while s0[heap_num] == 0:
            heap_num = np.random.randint(0,num_heaps)
        a0 = (heap_num, np.random.randint(1, s0[heap_num]+1))

        S, A, R, first_visit = generate_episode(policy=policy, s0=s0, a0=a0, Q=Q)

        if len(S) == 0:
            continue

        G = 0
        for t in range(len(R)-1, -1, -1):
            G = gamma * G + R[t]
            if first_visit[(S[t], A[t])] == t:

                if (S[t], A[t]) not in N:
                    N[(S[t], A[t])] = 0
                N[(S[t], A[t])] += 1

                if S[t] not in Q:
                    Q[S[t]] = {}
                if A[t] not in Q[S[t]]:
                    Q[S[t]][A[t]] = 0
                Q[S[t]][A[t]] = Q[S[t]][A[t]] + float((G-Q[S[t]][A[t]]))/float(N[(S[t], A[t])])


        policy.update(epsilon=1.0/float(i+1))

    return policy, Q

In [59]:
def nim_sum(s):
    xor = 0
    for heap in s:
        xor ^= heap
    return xor

In [60]:
def is_optimal(s, a):
    s = list(s)
    s[a[0]] -= a[1]
    return nim_sum(s) == 0

In [61]:
# Epsilon Greedy and exploring starts
policy, Q = monte_carlo_es()

print(Q)

100%|██████████| 100000/100000 [00:02<00:00, 42723.21it/s]

{(1, 0, 1): {(2, 1): 0.002674101046882119, (0, 1): 0.0}, (2, 0, 2): {(2, 1): 0.004707532051282045, (0, 2): 0.0, (0, 1): 0.0, (2, 2): 0.0}, (3, 4, 2): {(0, 1): 0.0, (2, 1): 0.0, (0, 3): 0.0, (2, 2): 0.0, (0, 2): 0.0, (1, 3): 1.0, (1, 4): 0.0, (1, 1): 0.0, (1, 2): 0.0}, (0, 1, 1): {(1, 1): 0.006410593242604174, (2, 1): 0.0}, (0, 2, 2): {(1, 1): 0.011321107784431121, (2, 1): 0.0, (2, 2): 0.0, (1, 2): 0.0}, (2, 2, 4): {(2, 2): 0.0, (1, 1): 0.0, (0, 2): 0.0, (2, 3): 0.0, (0, 1): 0.0, (1, 2): 0.0, (2, 4): 1.0, (2, 1): 0.0}, (2, 1, 3): {(1, 1): 0.008533747090768058, (2, 3): 0.0, (2, 2): 0.0, (0, 2): 0.0, (0, 1): 0.0, (2, 1): 0.0}, (5, 1, 3): {(0, 1): 0.0, (1, 1): 0.0, (2, 2): 0.0, (2, 3): 0.0, (0, 4): 0.0, (2, 1): 0.0, (0, 3): 1.0, (0, 5): 0.0, (0, 2): 0.0}, (0, 1, 0): {(1, 1): 0.7912420382165621}, (1, 2, 0): {(1, 1): 0.7338071065989841, (1, 2): 0.0, (0, 1): 0.0}, (2, 2, 1): {(2, 1): 0.6735862562634218, (0, 2): 0.0, (1, 1): 0.0, (0, 1): 0.0, (1, 2): 0.0}, (5, 2, 3): {(2, 1): 0.0, (0, 1): 0.0,

In [62]:
import itertools

def evaluate_on_winning_positions(policy, Q):
    fumbles = 0
    num_winning_pos = 0
    for s in itertools.product(range(initial_pebbles), repeat=num_heaps):
        if s == (0)*num_heaps: continue
        action = policy.final_policy(s, Q)
        if nim_sum(s) == 0: continue
        if not is_optimal(s, action):
            fumbles+=1
        num_winning_pos+=1
    return fumbles, num_winning_pos

In [63]:
fumbles, num_winning_pos = evaluate_on_winning_positions(policy, Q)
print('number of fumbles:', fumbles)
print('total number of winning positions:', num_winning_pos)
print('percentage of winning positions fumbled: {:.2f} %'.format(fumbles/num_winning_pos*100))

number of fumbles: 3
total number of winning positions: 106
percentage of winning positions fumbled: 2.83 %


In [64]:
policy, Q = monte_carlo_es(policy=SimpleGreedyPolicy())

print(Q)

100%|██████████| 100000/100000 [00:02<00:00, 45519.74it/s]

{(0, 1, 1): {(1, 1): 0.18447270370970417, (2, 1): 0.0}, (3, 1, 2): {(2, 1): 0.0, (1, 1): 0.0, (0, 2): 0.0, (2, 2): 0.0, (0, 1): 0.0, (0, 3): 0.0}, (3, 4, 4): {(2, 2): 0.0, (2, 3): 0.5360824742268041, (0, 3): 0.5454545454545454, (1, 2): 0.5409836065573769, (0, 1): 0.043478260869565244, (1, 4): 0.0, (2, 1): 0.0, (1, 3): 0.0, (0, 2): 0.0, (2, 4): 0.0, (1, 1): 0.0}, (5, 0, 0): {(0, 5): 1.0, (0, 1): 0.0, (0, 2): 0.0, (0, 4): 0.0, (0, 3): 0.0}, (0, 0, 2): {(2, 1): 0.0, (2, 2): 1.0}, (1, 1, 0): {(1, 1): 0.23993929080623722, (0, 1): 0.0}, (2, 2, 0): {(0, 1): 0.0004448398576512464, (1, 1): 0.0, (0, 2): 0.0, (1, 2): 0.0}, (3, 3, 0): {(1, 1): 0.0012825994014536133, (1, 2): 0.0, (0, 2): 0.0, (1, 3): 0.0, (0, 3): 0.0, (0, 1): 0.0}, (3, 5, 3): {(2, 3): 0.0, (2, 1): 0.0, (0, 2): 1.0, (1, 2): 0.0, (0, 3): 0.0, (1, 1): 0.0, (1, 4): 0.0, (0, 1): 0.0, (1, 5): 1.0, (2, 2): 1.0, (1, 3): 1.0}, (0, 2, 2): {(2, 1): 0.001820830298616179, (1, 2): 0.0, (1, 1): 0.0, (2, 2): 0.0}, (1, 2, 3): {(0, 1): 0.01807228915

In [65]:
fumbles, num_winning_pos = evaluate_on_winning_positions(policy, Q)
print('number of fumbles:', fumbles)
print('total number of winning positions:', num_winning_pos)
print('percentage of winning positions fumbled: {:.2f} %'.format(fumbles/num_winning_pos*100))

number of fumbles: 31
total number of winning positions: 106
percentage of winning positions fumbled: 29.25 %


In [ ]:
def generate_episode_no_es(policy, s0, Q):

    # init
    game = SimpleNim(s0)
    states = []
    actions = []
    rewards = []

    first_visit = {}
    index = 0

    # loop until end of episode
    while not game.is_game_over():
        # agent move
        curr_state = game.get_state()
        states.append(curr_state)
        curr_action = policy.policy(curr_state, Q)
        actions.append(curr_action)
        game.make_move(curr_action[0], curr_action[1])

        if (tuple(curr_state), curr_action) not in first_visit:
            first_visit[(tuple(curr_state), curr_action)] = index
        index+=1

        # opponent move
        play_optimal(game)

        if not game.is_game_over(): rewards.append(0)

    if game.winner() == 1:
        rewards.append(1)
    else:
        rewards.append(0)
    return states, actions, rewards, first_visit

In [71]:
# Monte Carlo Control with exploring starts
def monte_carlo_no_es(gamma=1, policy=EpsilonGreedyPolicy()):
    Q = {} # take default to be zero
    N = {}


    for i in tqdm(range(max_episodes)):
        policy.update(epsilon=1.0/float(i+1))
        s0 = np.random.randint(0, initial_pebbles+1, num_heaps)
        while all(s0 == 0):
            s0 = np.random.randint(0, initial_pebbles+1, num_heaps)

        S, A, R, first_visit = generate_episode_no_es(policy=policy, s0=s0, Q=Q)

        if len(S) == 0:
            continue

        G = 0
        for t in range(len(R)-1, -1, -1):
            G = gamma * G + R[t]
            if first_visit[(S[t], A[t])] == t:

                if (S[t], A[t]) not in N:
                    N[(S[t], A[t])] = 0
                N[(S[t], A[t])] += 1

                if S[t] not in Q:
                    Q[S[t]] = {}
                if A[t] not in Q[S[t]]:
                    Q[S[t]][A[t]] = 0
                Q[S[t]][A[t]] = Q[S[t]][A[t]] + float((G-Q[S[t]][A[t]]))/float(N[(S[t], A[t])])

    return policy, Q

In [72]:
policy, Q = monte_carlo_no_es()

print(Q)

100%|██████████| 100000/100000 [00:02<00:00, 47684.55it/s]

{(1, 1, 0): {(1, 1): 0.8888888888888888, (0, 1): 1.0}, (1, 1, 5): {(2, 1): 1.0}, (1, 0, 1): {(2, 1): 0.9184198713152749, (0, 1): 0.8636363636363634}, (2, 0, 2): {(2, 1): 0.8936407368297552}, (3, 1, 2): {(1, 1): 0.8573178199632573}, (4, 1, 5): {(0, 1): 0.7594594594594606, (2, 1): 0.0}, (5, 2, 5): {(1, 1): 0.6807444523979957}, (5, 3, 5): {(1, 1): 0.52355460385439}, (4, 1, 4): {(0, 1): 0.8281580804516571}, (0, 1, 1): {(1, 1): 1.0}, (0, 2, 2): {(2, 1): 1.0, (1, 1): 1.0}, (0, 3, 3): {(1, 1): 1.0}, (0, 5, 4): {(2, 1): 1.0}, (2, 2, 0): {(0, 1): 1.0}, (3, 2, 5): {(0, 1): 1.0}, (3, 0, 1): {(0, 1): 1.0}, (1, 3, 0): {(0, 1): 1.0}, (1, 5, 0): {(0, 1): 1.0}, (3, 3, 0): {(1, 1): 1.0}, (4, 4, 0): {(1, 1): 1.0}, (5, 5, 0): {(1, 1): 1.0}, (3, 1, 5): {(2, 1): 1.0}, (2, 3, 1): {(0, 1): 0.909550671075666, (1, 1): 0.0}, (4, 4, 1): {(1, 1): 0.7474198804997284}, (4, 4, 2): {(2, 1): 0.49346405228758183}, (4, 5, 2): {(1, 1): 1.0}, (0, 4, 4): {(1, 1): 1.0}, (5, 4, 5): {(1, 1): 1.0}, (5, 4, 3): {(1, 1): 1.0}, (4

In [73]:
fumbles, num_winning_pos = evaluate_on_winning_positions(policy, Q)
print('number of fumbles:', fumbles)
print('total number of winning positions:', num_winning_pos)
print('percentage of winning positions fumbled: {:.2f} %'.format(fumbles/num_winning_pos*100))

number of fumbles: 82
total number of winning positions: 106
percentage of winning positions fumbled: 77.36 %
